In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#read in dcra_cama merged data csv
df = pd.read_csv("cleaned_merged_data5.csv", low_memory=False)

In [ ]:
df.head()

In [ ]:
#Max number of review cycles for a ssl
#maxRC = df.groupby(["ssl"],  sort= False)['ReviewCycle'].max()
#maxRC = maxRC.reset_index()

In [ ]:
#Mean number of review cycles for a ssl rounded to nearest integer
maxRC = df.groupby(["ssl"],  sort= False)['ReviewCycle'].aggregate(np.mean).round(0)
maxRC = maxRC.reset_index()

In [ ]:
maxRC.shape
maxRC.head()

In [ ]:
#mean time taken per ssl
#time = df.groupby(["ssl"],  sort= False)['elapsed_workdays'].aggregate(np.mean).round(3)
#time = time.reset_index()

In [ ]:
#print(time.shape)
#time.head()

In [ ]:
print(df.columns)
df.head()

In [ ]:
#make a new df combining same depts with varying reviewCycle
dfnew = df.groupby(['ssl', 'GroupName', 'Ward','use_type', 'PRICE', 'QUALIFIED', 'EYB', 'SALE_NUM', 'LIVING_GBA', 'USECODE',
       'LANDAREA' ], sort=False)['elapsed_workdays'].aggregate(np.sum)

dfnew=dfnew.reset_index()

In [ ]:
#mean time taken per ssl for newdf
time = dfnew.groupby(["ssl"],  sort= False)['elapsed_workdays'].aggregate(np.mean).round(3)
time = time.reset_index()
time.head()

In [ ]:
time.shape

In [ ]:
print(dfnew.shape)
dfnew.head(30)

In [ ]:
#drop total elapsed_workdays column because we are using mean elapsed_workdays per ssl as outcome
dfnew = dfnew.drop(['elapsed_workdays'], axis=1)

In [ ]:
#create dummies for all agencies
agencies = pd.get_dummies(dfnew.GroupName)
agencies.shape

In [ ]:
#concat the two dataframes
dfnew = pd.concat([dfnew,agencies], axis=1)
dfnew.columns

In [ ]:
#groupby the agencies across the same SSLs to show all the agencies permits from an SSL have been to 
dfnew = dfnew.groupby(['ssl', 'Ward','use_type', 'PRICE', 'QUALIFIED', 'EYB', 'SALE_NUM', 'LIVING_GBA', 'USECODE',
       'LANDAREA' ], sort=False)['CFA Review', 'Chinatown Review', 'DC Water Review', 'DDOT Review',
       'DOEE', 'DOH Review', 'EISF Review', 'Electrical Review',
       'Elevator Review', 'Energy Review', 'Fire Review', 'Green Review',
       'HPRB Review', 'Mechanical Review', 'NCPC Review', 'Plumbing Review',
       'Structural Review', 'WMATA Review', 'White House Review',
       'Zoning Review'].aggregate(np.sum)

dfnew = dfnew.reset_index()

In [ ]:
#NOT USING BUT GOOD CODE
#pivot the dataframe on sister Agencies denoted by 'groupname' to create columns for all agency types
#dfnew = pd.pivot_table(dfnew, values = 'elapsed_workdays', index=['ssl', 'Ward', 'use_type', 'PRICE', 'QUALIFIED', 'EYB', 'SALE_NUM', 'LIVING_GBA', 'USECODE',
       #'LANDAREA'], columns = 'GroupName').reset_index().fillna(0)


In [ ]:
dfnew.head()

In [ ]:
dfnew.shape

In [ ]:
len(pd.unique(dfnew.ssl))

In [ ]:
#find duplicate rows based on 'ssl'
dfnew[dfnew.duplicated(['ssl'], keep=False)]

In [ ]:
#dropping duplicates by index because few duplicates
dfnew.drop([133,404,465,571,887,1296], inplace=True)

In [ ]:
dfnew.shape

In [ ]:
#merge the newdf dataframe with max reviewCycle db and total time taken for a permit db, keeping ssls from newdf only
dfnew = pd.merge(dfnew, maxRC, on='ssl', how='left')
dfnew = pd.merge(dfnew, time, on='ssl', how='left')
#dat = dat.drop(['Ward'], axis=1)

In [ ]:
print(dfnew.shape)
dfnew.head()

In [ ]:
dfnew.groupby('SALE_NUM').size()

In [ ]:
#dfnew.groupby('USECODE').size()
dfnew.columns

In [ ]:
#feature and target
y = dfnew['elapsed_workdays']
X = dfnew.drop(['ssl', 'elapsed_workdays'], axis=1)


In [ ]:
#log-transform variables with large numeric values 
import math
X['PRICE'] = X['PRICE'].apply(lambda x: math.log(x,10) if x > 0 else x).round(3)
X['LIVING_GBA'] = X['LIVING_GBA'].apply(lambda x: math.log(x,10) if x > 0 else x).round(3)
X['LANDAREA'] = X['LANDAREA'].apply(lambda x: math.log(x,10) if x > 0 else x).round(3)

In [ ]:
#Change EYB to elapsed years from construction to make it continuous

X['EYB'] = 2018 - X['EYB']

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
CV = [ 'Ward','use_type', 'QUALIFIED', 'SALE_NUM', 'USECODE' ]
for v in CV:
    X[v] = lb.fit_transform(X[v])

In [ ]:
X.head()

In [ ]:
dfnew.groupby(['elapsed_workdays']).size()

In [ ]:
#dfnew.groupby(['elapsed_workdays']).size()
time_median = dfnew['elapsed_workdays'].median()

In [ ]:
#y.groupby("elapsed_workdays").size()
np.percentile(y, [5, 25, 50, 75, 95])

In [ ]:
#replace outliers beore 5th percentile and beyond 95th percentile by median elapsed_workdays value
y = y.apply(lambda x:time_median if x > 66 else x)


In [ ]:
y = y.apply(lambda x:time_median if x < 6 else x)

In [ ]:
np.percentile(y, [25, 50, 75, 95])

In [ ]:
# Suppress chaining warning
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
#making new target array based on the mean time a permit takes by weeks
y7 = y.apply(lambda x: 1 if x > 7 else 0)
y14 = y.apply(lambda x: 1 if x > 14 else 0)
y21 = y.apply(lambda x: 1 if x > 21 else 0)
y30 = y.apply(lambda x: 1 if x > 30 else 0)

#50th percentile
#y['elapsed_workdays_10'] = y['elapsed_workdays'].apply(lambda x: 1 if x > 10 else 0)

In [ ]:
y7.groupby(y7).size()

In [ ]:
y14.groupby(y14).size()

In [ ]:
y21.groupby(y21).size()

In [ ]:
y30.groupby(y30).size()

Based on the distribution of data in the target array, it looks like 21 and 30 are good thresholds. 

In [ ]:
import matplotlib.pyplot as plt   #Data visualisation libraries 
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc

In [ ]:
#split the dataset into test and trains sets

X_train, X_test, y21_train, y21_test = train_test_split(X, y21, test_size=0.3, random_state=101)
X_train, X_test, y30_train, y30_test = train_test_split(X, y30, test_size=0.3, random_state=101)

In [ ]:
#LOGIT MODEL
from sklearn.linear_model import LogisticRegression

In [ ]:
logisticRegr = LogisticRegression()

In [ ]:
l21 = logisticRegr.fit(X_train, np.ravel(y21_train))
predictions21 = l21.predict(X_test)

In [ ]:
l30 = logisticRegr.fit(X_train, np.ravel(y30_train))
predictions30 = l30.predict(X_test)

In [ ]:
print('accuracy for logistic regression model 21-days: %f' %accuracy_score(np.ravel(y21_test),predictions21))
print('accuracy for logistic regression model 30-days: %f' %accuracy_score(np.ravel(y30_test),predictions30))

In [ ]:
#MAKE ROC FOR LOGIT Y=21 DAYS

logit = LogisticRegression()
logit.fit(X_train, y21_train)
print('Training score: ', logit.score(X_test, y21_test))
y_pred = logit.predict(X_test)

In [ ]:
# true positives and false positives for logit for 21 days

false_positive_rate, true_positive_rate, thresholds = roc_curve(y21_test, y_pred, pos_label=1)
rates_logit = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc_logit = auc(rates_logit['fpr'], rates_logit['tpr'])
print('AUC: ', roc_auc_logit)

In [ ]:
plt.plot(rates_logit.fpr, rates_logit.tpr, 'b', label = 'logit = %0.3f' % roc_auc_logit)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic for Logit')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
#ROC FOR 30 DAYS
logit = LogisticRegression()
logit.fit(X_train, y30_train)
print('Training score: ', logit.score(X_test, y30_test))
y_pred = logit.predict(X_test)

In [ ]:
# true positives and false positives for logit for 30 days

false_positive_rate, true_positive_rate, thresholds = roc_curve(y30_test, y_pred, pos_label=1)
rates = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc = auc(rates['fpr'], rates['tpr'])
print('AUC: ', roc_auc)


plt.plot(rates.fpr, rates.tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic for Logit')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score, validation_curve, learning_curve, train_test_split

In [ ]:
# create knn model with k=1
k1 = KNeighborsClassifier(n_neighbors = 1)

#k17 = k1.fit(X_train, np.ravel(y7_train))
k121 = k1.fit(X_train, np.ravel(y21_train))
k130 = k1.fit(X_train, np.ravel(y30_train))

#Y_k17_predict = k17.predict(X_test)
Y_k121_predict = k121.predict(X_test)
Y_k130_predict = k130.predict(X_test)

#print('k-NN accuracy for test set k=1: %f' % accuracy_score(y7_test,Y_k17_predict))
print('k-NN accuracy for test set k=1 and 21 days data: %f' % accuracy_score(y21_test,Y_k121_predict))
print('k-NN accuracy for test set k=1 and 30 days data: %f' % accuracy_score(y30_test,Y_k130_predict))

In [ ]:
# create knn model with k=7
k7 = KNeighborsClassifier(n_neighbors = 7)

#k77 = k7.fit(X_train, np.ravel(y7_train))
k721 = k7.fit(X_train, np.ravel(y21_train))
k730 = k7.fit(X_train, np.ravel(y30_train))

#Y_k77_predict = k77.predict(X_test)
Y_k721_predict = k721.predict(X_test)
Y_k730_predict = k730.predict(X_test)

#print('k-NN accuracy for test set k=7: %f' % accuracy_score(y7_test,Y_k77_predict))
print('k-NN accuracy for test set k=7 and 21 days data: %f' % accuracy_score(y21_test,Y_k721_predict))
print('k-NN accuracy for test set k=7 and 30 days data: %f' % accuracy_score(y30_test,Y_k730_predict))

In [ ]:
# create knn model with k=15
k15 = KNeighborsClassifier(n_neighbors = 15)

#k15_7 = k15.fit(X_train, np.ravel(y7_train))
k15_21 = k15.fit(X_train, np.ravel(y21_train))
k15_30 = k15.fit(X_train, np.ravel(y30_train))

#Y_k15_7_predict = k15_7.predict(X_test)
Y_k15_21_predict = k15_21.predict(X_test)
Y_k15_30_predict = k15_30.predict(X_test)

#print('k-NN accuracy for test set k=15: %f' % accuracy_score(y7_test,Y_k15_7_predict))
print('k-NN accuracy for test set k=15 and 21 days data: %f' % accuracy_score(y21_test,Y_k15_21_predict))
print('k-NN accuracy for test set k=15 and 30 days data: %f' % accuracy_score(y30_test,Y_k15_30_predict))

In [ ]:
#10-fold cross validation
#k=1
#score_k1_7 = cross_val_score(k17,X, np.ravel(y7), cv = 10)
score_k1_21 = cross_val_score(k121,X, np.ravel(y21), cv = 10)
score_k1_30 = cross_val_score(k130,X, np.ravel(y30), cv = 10)

#print('the avg score for k1_7 model is: %f' %np.mean(score_k1_7))
print('the avg score for k1_21 model is: %f' %np.mean(score_k1_21))
print('the avg score for k1_30 model is: %f' %np.mean(score_k1_30))


In [ ]:
#k=7
#score_k7_7 = cross_val_score(k77,X, np.ravel(y7), cv = 10)
score_k7_21 = cross_val_score(k721,X, np.ravel(y21), cv = 10)
score_k7_30 = cross_val_score(k730,X, np.ravel(y30), cv = 10)

#print('the avg score for k7_7 model is: %f' %np.mean(score_k7_7))
print('the avg score for k7_21 model is: %f' %np.mean(score_k7_21))
print('the avg score for k7_30 model is: %f' %np.mean(score_k7_30))

In [ ]:
#k=15
#score_k15_7 = cross_val_score(k15_7,X, np.ravel(y7), cv = 10)
score_k15_21 = cross_val_score(k15_21,X, np.ravel(y21), cv = 10)
score_k15_30 = cross_val_score(k15_30,X, np.ravel(y30), cv = 10)

#print('the avg score for k15_7 model is: %f' %np.mean(score_k15_7))
print('the avg score for k15_21 model is: %f' %np.mean(score_k15_21))
print('the avg score for k15_30 model is: %f' %np.mean(score_k15_30))


In [ ]:
#validation curve for cv=10 using k15 fitted model for k values 6 to 42
k_range = np.arange(6,55,3 )
train_scores, test_scores = validation_curve(k15_21, X, y21, param_name='n_neighbors',
                                            param_range=k_range, cv=10)

In [ ]:
train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
print(test_mean)
print(train_mean)

In [ ]:
score = np.arange(101)
max_depth = np.arange(6,55,3)
plt.plot(max_depth, train_mean, label='Train')
plt.plot(max_depth, test_mean, label='Test')
plt.xlabel('Maximum K')
plt.ylabel('Score')
plt.title('KNN Validation Curve')
plt.legend()
plt.show()

In [ ]:
#looks best for k=10 for y=21 days/3 weeks target variable

#k=10
k10 = KNeighborsClassifier(n_neighbors = 10)
k10.fit(X_train, np.ravel(y21_train))
Y_k10_predict = k10.predict(X_test)
print('k-NN accuracy for test set k=10: %f' % accuracy_score(y21_test,Y_k10_predict))

score_k10 = cross_val_score(k10, X, np.ravel(y21), cv = 10)
print(score_k10)
print('the avg score for k10 model is: %f' %np.mean(score_k10))

In [ ]:
#k=50
k50 = KNeighborsClassifier(n_neighbors = 50)
k50.fit(X_train, np.ravel(y21_train))
Y_k50_predict = k50.predict(X_test)
print('k-NN accuracy for test set k=50: %f' % accuracy_score(y21_test,Y_k50_predict))

score_k50 = cross_val_score(k50, X, np.ravel(y21), cv = 10)
print(score_k50)
print('the avg score for k50 model is: %f' %np.mean(score_k50))

In [ ]:
# true positives and false positives for knn y-21 days model and k=10
false_positive_rate, true_positive_rate, thresholds = roc_curve(y21_test, Y_k10_predict, pos_label=1)
rates_knn = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc_knn = auc(rates_knn['fpr'], rates_knn['tpr'])
print('AUC: ', roc_auc_knn)

In [ ]:
plt.plot(rates_knn.fpr, rates_knn.tpr, 'g', label = 'KNN = %0.3f' % roc_auc_knn)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic k-10 y-21days')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# true positives and false positives for knn y-21 days model and k=50
false_positive_rate, true_positive_rate, thresholds = roc_curve(y21_test, Y_k50_predict, pos_label=1)
rates_knn = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc = auc(rates_knn['fpr'], rates_knn['tpr'])
print('AUC: ', roc_auc)

plt.plot(rates_knn.fpr, rates_knn.tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic k-50 y-21days')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
#Gaussian NB
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()
#g7 = gnb.fit(X, y7)
g21 = gnb.fit(X, y21)
g30 = gnb.fit(X, y30)

In [ ]:
#print(np.mean(cross_val_score(g7, X, y7, cv=10)))
print(np.mean(cross_val_score(g21, X, y21, cv=10)))
print(np.mean(cross_val_score(g30, X, y30, cv=10)))

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y21_train)
print('Training score: ', gnb.score(X_test, y21_test))
y_pred = gnb.predict(X_test)

In [ ]:
# true positives and false positives for GNB 21 days
false_positive_rate, true_positive_rate, thresholds = roc_curve(y21_test, y_pred, pos_label=1)
rates_gnb = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc_gnb = auc(rates_gnb['fpr'], rates_gnb['tpr'])
print('AUC_GNB: ', roc_auc_gnb)

plt.plot(rates_gnb.fpr, rates_gnb.tpr, 'r', label = 'GNB = %0.3f' % roc_auc_gnb)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic GNB 21 days')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
#decision tree
import graphviz
from sklearn import tree
import matplotlib.pyplot as plt
%matplotlib inline 
import graphviz

In [ ]:
model = tree.DecisionTreeClassifier(criterion = "gini", max_depth=4)

#m7 = model.fit(X_train,y7_train)
m21 = model.fit(X_train,y21_train)
m30 = model.fit(X_train,y30_train)

In [ ]:
#prediction7 = m7.predict(X_test)
prediction21 = m21.predict(X_test)
prediction30 = m30.predict(X_test)

#print('DT accuracy for test set 7-days: %f' % accuracy_score(y7_test,prediction7))
print('DT accuracy for test set 21-days: %f' % accuracy_score(y21_test,prediction21))
print('DT accuracy for test set 30-days: %f' % accuracy_score(y30_test,prediction30))

In [ ]:
#print(np.mean(cross_val_score(model, X, ybin, cv=3)))
#print(np.mean(cross_val_score(model, X, ybin, cv=5)))

#print(np.mean(cross_val_score(m7, X, y7, cv=10)))
print(np.mean(cross_val_score(m21, X, y21, cv=10)))
print(np.mean(cross_val_score(m30, X, y30, cv=10)))


In [ ]:
# true positives and false positives for DT y-21 days model
false_positive_rate, true_positive_rate, thresholds = roc_curve(y21_test, prediction21, pos_label=1)
rates_dtm = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc_dtm = auc(rates_dtm['fpr'], rates_dtm['tpr'])
print('AUC DTM: ', roc_auc_dtm)

plt.plot(rates_dtm.fpr, rates_dtm.tpr, 'c', label = 'AUC = %0.3f' % roc_auc_dtm)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic DT 21 days')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# Visualize the tree model for 21 and 30 days in separate file
import pydotplus

dot_data21 = tree.export_graphviz(m21, out_file=None, feature_names=X.columns, filled=True,rounded=True)                                
graph21 = graphviz.Source(dot_data21)
graph21.render("SSL data tree")

pydot_graph = pydotplus.graph_from_dot_data(dot_data21)
pydot_graph.write_png('original_tree.png')
#graph21.savefig('tree.png')

In [ ]:
# plot variable importance for 14-days model
var_imp = pd.DataFrame({'Variable': X.columns, 
                        'Importance': m21.feature_importances_})
var_imp = var_imp.sort_values(by='Importance', ascending=False)
var_imp = var_imp[:-18].sort_values(by='Importance')
var_imp

In [ ]:
fig1 = plt.figure(figsize=(5, 3.5))
plt.barh(var_imp['Variable'], var_imp['Importance'])
plt.xlabel('Proportion')
plt.ylabel('Variable')
plt.title('Variable Importance')
plt.show()
fig1.savefig('vim.png')

In [ ]:
# random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

In [ ]:

rfmodel = RandomForestClassifier(n_estimators=100, criterion = "entropy",max_depth=5, random_state=0)
#rf7 = rfmodel.fit(X, y7)
rf21 = rfmodel.fit(X, y21)
rf30 = rfmodel.fit(X, y30)

In [ ]:
#print(np.mean(cross_val_score(rf7, X, y7, cv=10)))
print(np.mean(cross_val_score(rf21, X, y21, cv=10)))
print(np.mean(cross_val_score(rf30, X, y30, cv=10)))

In [ ]:
rfm = RandomForestClassifier(n_estimators=100, criterion = "entropy",random_state=0)
rfm.fit(X_train, y21_train)
print('Training score: ', rfm.score(X_test, y21_test))
y_pred = rfm.predict(X_test)

In [ ]:
# true positives and false positives for rfm y-21 days model
false_positive_rate, true_positive_rate, thresholds = roc_curve(y21_test, y_pred, pos_label=1)
rates_rfm = pd.DataFrame(dict(fpr=false_positive_rate, tpr=true_positive_rate))
roc_auc_rfm = auc(rates_rfm['fpr'], rates_rfm['tpr'])
print('AUC RFM: ', roc_auc_rfm)

plt.plot(rates_rfm.fpr, rates_rfm.tpr, 'y', label = 'AUC = %0.3f' % roc_auc_rfm)
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Receiver Operating Characteristic RFC 21 days')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
estimator = rfmodel.estimators_[5]

In [ ]:
export_graphviz(estimator, out_file='rftree.dot', 
                feature_names = X.columns,
                
                rounded = True, proportion = False, 
                precision = 2, filled = True)


In [ ]:
# Visualize the tree in separate file
dot_data = RandomForestClassifier.export_graphviz(model, out_file=None, feature_names=X.columns)
graph = graphviz.Source(dot_data)
graph.render("cama")
graph

Combine all ROCs in one

In [ ]:
fig1 = plt.figure(figsize=(6, 3.5))


plt.plot(rates_knn.fpr, rates_knn.tpr, 'g', label = 'KNN = %0.3f' % roc_auc_knn)

plt.plot(rates_gnb.fpr, rates_gnb.tpr, 'r', label = 'GNB = %0.3f' % roc_auc_gnb)
plt.plot(rates_logit.fpr, rates_logit.tpr, 'b', label = 'logit = %0.3f' % roc_auc_logit)

plt.plot(rates_dtm.fpr, rates_dtm.tpr, 'c', label = 'DTM = %0.3f' % roc_auc_dtm)

#plt.plot(rates_rfm.fpr, rates_rfm.tpr, 'y', label = 'RFM = %0.3f' % roc_auc_rfm)
#ax = plt.add_axes([0,0,1,1])
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC for t=21 days')
plt.legend(loc = 'lower right')
plt.show()
fig1.savefig('ROC2.png', bbox_inches='tight')